In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("google/embeddinggemma-300m")

W1122 12:59:56.769000 8340 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [ ]:
print(model[0].auto_model)

In [ ]:
model[0].auto_model.config

In [19]:
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 2048, 'do_lower_case': False, 'architecture': 'Gemma3TextModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 3072, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
  (3): Dense({'in_features': 3072, 'out_features': 768, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
  (4): Normalize()
)


In [ ]:
model.tokenizer

In [9]:
vec = model.encode_query("Whenever I want some pizza")
vec

array([-1.48952082e-01,  1.79727320e-02, -1.86689552e-02,  2.08280981e-02,
        8.54775659e-04,  3.04947458e-02, -2.11084448e-02,  2.53285025e-03,
        4.60965838e-03,  3.31667103e-02,  3.35762054e-02, -4.04717848e-02,
        4.62193936e-02, -6.29886165e-02, -6.44645654e-03,  6.83294460e-02,
        3.23498733e-02, -3.47967562e-03, -3.17044519e-02, -4.37740795e-02,
        2.60711499e-02, -1.52124921e-02, -1.11142872e-02, -7.33840279e-03,
        8.28226376e-03,  6.05128966e-02,  7.85877742e-03, -7.37313228e-03,
        2.03651618e-02,  4.23159413e-02,  5.15062846e-02,  3.40949069e-03,
        1.65480878e-02,  4.06718031e-02,  1.96179245e-02, -1.22249024e-02,
        1.06935047e-01, -6.48506060e-02,  2.26615053e-02, -4.86925915e-02,
       -4.76503596e-02,  6.90020323e-02,  3.32487673e-02, -2.84347050e-02,
        2.66051237e-02, -9.81845427e-04,  9.61757265e-03, -2.47431919e-02,
       -1.97705477e-02, -4.05615866e-02, -1.38379484e-02,  9.70601384e-03,
        2.87891366e-03, -

In [18]:
import torch 

model[1].forward(
    {
        "token_embeddings":torch.randn(7, 768),
        "attention_mask": torch.ones(7)
    })

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [27]:
text = "Whenever I want some pizza"

# Tokenize
inputs = model.tokenizer(text, return_tensors="pt").to("cuda")

# Run transformer (module 0)
transformer = model[0]
out = transformer(inputs)
   # shape: (1, seq_len, 768)
print("Transformer output:", out["token_embeddings"])

Transformer output: tensor([[[ 2.8601,  1.0641, -1.0108,  ..., -1.3731,  0.8435,  2.1362],
         [ 0.4422, -1.5233, -1.0430,  ..., -0.7149,  0.3795,  0.7050],
         [-0.6354, -0.1884, -0.6559,  ..., -1.3086, -0.4310, -0.2763],
         ...,
         [-0.4821,  1.8441, -1.2279,  ..., -1.4918,  0.5263, -0.7490],
         [-0.8325,  0.2792, -0.0717,  ...,  2.2132, -1.7570, -0.5148],
         [-0.2945,  6.4030, -2.1014,  ..., -2.3982, -1.7439, -0.6997]]],
       device='cuda:0', grad_fn=<MulBackward0>)


In [29]:
pooling = model[1]

features = {
    "token_embeddings": out["token_embeddings"],
    "attention_mask": out["attention_mask"]
}

pooled_features = pooling(features)
pooled = pooled_features["sentence_embedding"]    # shape: (1, 768)
print("Pooling output:", pooled)

Pooling output: tensor([[ 1.9516e-01,  1.4175e+00, -9.9269e-01,  1.0643e+00, -3.3519e-01,
         -5.8965e-01, -1.0917e-01,  3.3590e-02, -4.7268e-01, -7.1670e-01,
         -5.6630e-02,  1.2974e-01,  8.0423e-01,  9.2162e-01,  3.4562e-01,
         -5.0482e-02,  1.2439e-02,  4.6458e-02, -7.5574e-02, -7.8400e-01,
          7.4679e-01,  4.8446e-01,  2.9131e-01,  5.0462e-01, -1.0403e+00,
          1.0466e+00,  1.2124e+00, -2.7889e-01, -1.1571e+00, -4.6625e-01,
         -3.9170e-01, -2.2883e-01, -2.0950e+00,  3.7477e-01, -1.9718e-02,
         -2.3489e-01, -1.1335e-01, -9.9005e-02, -2.8402e-01, -1.2804e+01,
          2.0867e-01,  3.7815e-01, -6.3674e-01, -3.0808e-01,  8.0453e-02,
          3.2593e-01,  3.0186e-01, -6.9296e-01,  2.6378e-02,  3.1440e-01,
          5.4629e-01,  2.9805e-01, -8.5997e-01, -3.8386e-01, -3.8419e-01,
         -1.5773e-01, -7.4994e-01,  8.8135e-01, -2.4473e-01,  7.3564e-01,
          1.5352e-01,  1.9692e-01, -4.3587e-01,  7.2615e-01, -6.1705e-01,
         -2.2686e-01, 

In [41]:
dense1 = model[2]


out_dense1 = dense1({
    "sentence_embedding": pooled
})
print("Dense1 output:", out_dense1["sentence_embedding"])   # (1, 768

Dense1 output: tensor([[-3.6195, -3.8176, -5.5409,  ...,  0.6482, -0.5006,  1.9404]],
       device='cuda:0', grad_fn=<MmBackward0>)


In [54]:
vars(dense1.linear)

{'training': False,
 '_parameters': {'weight': Parameter containing:
  tensor([[-0.0248,  0.0440, -0.1624,  ...,  0.0403, -0.1545,  0.1004],
          [-0.0503,  0.3225,  0.1328,  ..., -0.1440,  0.0932, -0.0687],
          [ 0.1091,  0.2012,  0.0360,  ..., -0.0451,  0.0322,  0.1307],
          ...,
          [-0.0105,  0.0329, -0.0861,  ...,  0.0136, -0.0926,  0.0940],
          [-0.0987,  0.0773,  0.1778,  ...,  0.0052,  0.1620, -0.1392],
          [-0.0597,  0.0600,  0.0995,  ...,  0.0411,  0.1798,  0.0289]],
         device='cuda:0', requires_grad=True),
  'bias': None},
 '_buffers': {},
 '_non_persistent_buffers_set': set(),
 '_backward_pre_hooks': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_hooks_with_kwargs': OrderedDict(),
 '_forward_hooks_always_called': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_

In [43]:
dense2 = model[3]


out_dense2 = dense2({
    "sentence_embedding": out_dense1["sentence_embedding"]
})
print("Dense2 output:", out_dense2["sentence_embedding"])   # (1, 768

Dense2 output: tensor([[-1.0408e+02, -4.9950e+00,  8.7484e+00,  4.8262e+00, -1.8623e-01,
          8.6964e+00, -3.1117e+01,  1.8455e+01,  1.6998e+01, -9.9459e+00,
         -5.7335e+00, -3.0252e+01,  3.6569e+01, -1.6661e+01,  4.0428e+01,
          2.2467e+01,  1.6820e+01, -3.4263e+01, -2.6193e+01,  2.6479e+00,
          1.0658e+01, -1.6332e+01,  6.9657e+00,  3.2710e-01,  3.1198e+00,
          1.8477e+01, -5.6170e+00, -2.5274e-01,  4.3680e+00, -8.6656e+00,
          4.1029e+01,  1.6641e+00,  1.9013e+01,  7.9236e+00,  5.2815e+00,
          2.2317e+01,  3.7866e+01, -3.9196e+01,  3.5293e+01, -1.7790e+01,
         -4.5564e+01,  4.2999e+01,  9.4261e+00, -2.0259e+01,  2.3210e+01,
          5.4496e+00, -5.9906e+00, -1.9547e+01, -9.9402e+00, -1.3917e+01,
          5.7495e+00,  1.5366e+01, -1.1385e+01, -2.0611e+01, -2.4795e+01,
         -2.7823e+01, -1.5134e+01,  1.2376e+01, -6.1311e+00,  1.6711e+01,
         -3.7810e+01, -4.8631e+00, -6.9120e+00,  1.0702e+01,  3.5922e+01,
         -2.7022e+01,  

In [55]:
vars(dense2.linear)

{'training': False,
 '_parameters': {'weight': Parameter containing:
  tensor([[ 0.0644, -0.0442,  0.0584,  ..., -0.0710, -0.0331,  0.1013],
          [-0.0256,  0.0220, -0.0541,  ...,  0.0673,  0.0118,  0.0433],
          [-0.0135,  0.1833,  0.0783,  ...,  0.1446, -0.0160,  0.0841],
          ...,
          [ 0.0324,  0.1872,  0.0370,  ..., -0.0647, -0.1224,  0.0687],
          [-0.1058,  0.1690,  0.1313,  ...,  0.0142,  0.0178, -0.0441],
          [-0.0646,  0.0513, -0.0635,  ...,  0.1068,  0.0721, -0.0533]],
         device='cuda:0', requires_grad=True),
  'bias': None},
 '_buffers': {},
 '_non_persistent_buffers_set': set(),
 '_backward_pre_hooks': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_hooks_with_kwargs': OrderedDict(),
 '_forward_hooks_always_called': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_

In [46]:
normed = model[4](
    out_dense2)
normed

{'sentence_embedding': tensor([[-1.7511e-01, -8.4041e-03,  1.4719e-02,  8.1202e-03, -3.1333e-04,
           1.4632e-02, -5.2355e-02,  3.1051e-02,  2.8599e-02, -1.6734e-02,
          -9.6467e-03, -5.0898e-02,  6.1527e-02, -2.8033e-02,  6.8020e-02,
           3.7801e-02,  2.8300e-02, -5.7648e-02, -4.4070e-02,  4.4551e-03,
           1.7933e-02, -2.7479e-02,  1.1720e-02,  5.5034e-04,  5.2491e-03,
           3.1088e-02, -9.4506e-03, -4.2524e-04,  7.3491e-03, -1.4580e-02,
           6.9031e-02,  2.7999e-03,  3.1989e-02,  1.3331e-02,  8.8861e-03,
           3.7549e-02,  6.3709e-02, -6.5948e-02,  5.9381e-02, -2.9932e-02,
          -7.6661e-02,  7.2346e-02,  1.5860e-02, -3.4086e-02,  3.9051e-02,
           9.1690e-03, -1.0079e-02, -3.2889e-02, -1.6725e-02, -2.3416e-02,
           9.6737e-03,  2.5853e-02, -1.9155e-02, -3.4679e-02, -4.1717e-02,
          -4.6813e-02, -2.5463e-02,  2.0823e-02, -1.0316e-02,  2.8117e-02,
          -6.3616e-02, -8.1822e-03, -1.1629e-02,  1.8007e-02,  6.0439e-02,
   

In [48]:
vars(model[4])

{'training': False,
 '_parameters': {},
 '_buffers': {},
 '_non_persistent_buffers_set': set(),
 '_backward_pre_hooks': OrderedDict(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_hooks_with_kwargs': OrderedDict(),
 '_forward_hooks_always_called': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_forward_pre_hooks_with_kwargs': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_load_state_dict_post_hooks': OrderedDict(),
 '_modules': {}}